In [1]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct

import warnings
from sklearn.exceptions import DataConversionWarning

# Ignore all warnings from scikit-learn
warnings.filterwarnings("ignore", module="sklearn")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
# Add more categories if needed



In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 5,
    "exp_name": ct.generate_readable_short_id("Real30_CTfix"), #"main_run5_cv5_21data_100trees_40opt_fast",
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 5,
    "plot": False,

    "calib_methods": [
                      "RF_d","RF_opt", "RF_large", 
                      "Platt", "ISO", "Beta", "VA",
                      "CT", "CT_minleaf", "PPA", "Rank"
                      ],

    "calib_method_colors": {
        "RF_d": "blue", 
        "RF_opt": "orange", 
        "RF_large": "red",
        "CT": "slategray",
        "CT_minleaf": "black",
        "Platt": "Brown", 
        "ISO": "purple", 
        "Beta": "magenta", 
        "PPA": "olive", 
        "VA": "gray",
        "Rank": "silver"     
    },

    # "calib_methods": ["RF_opt", "RF_large",
    #                   "DT", "LR", "SVM", "NN", "GNB"          
    #                   ],


    "metrics": ["acc", "brier", "ece", "logloss", "time"],

    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth":  np.arange(2, 100).tolist(), # [None, 5, 10, 15, 20, 30, 50], #
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf":  np.arange(1, 200).tolist(),
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False]
                    },
    
    "oob": False,
    "laplace": 0, # if set to 1, OOB must be false
    "curt_v": np.arange(1, 200).tolist(),

    "exp_key": "data_name",
    "exp_values": [
                  "cm1",
                  "datatrieve",
                  "kc1_class_level_defectiveornot",
                  "kc1",
                  "kc2",
                  "kc3",
                  "pc1",
                  "spect",
                  "spectf",
                  "vertebral", 
                  "wilt",
                  "parkinsons", 
                  "heart",
                  "wdbc",
                  "bank", 
                  "ionosphere", 
                  "HRCompetencyScores",
                  "spambase", 
                  "QSAR", 
                  "diabetes", 
                  "breast", 
                  "SPF",
                  "hillvalley",
                  "pc4",
                  "scene",
                  "Sonar_Mine_Rock_Data",
                  "Customer_Churn",
                  "jm1",
                  "eeg",
                  "phoneme",


                  # "madelon",
                  # "nomao",
                  ]
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param cm1 done
exp_param datatrieve done


In [4]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=True, 
                                    std=True)
ct.save_metrics_to_csv(tables, params["metrics"], params['exp_name'])

In [5]:
tables["time"].round(4)


,RF_d,RF_opt,RF_large,Platt,ISO,Beta,VA,CT,CT_minleaf,PPA,Rank
Data,,,,,,,,,,,
cm1,0.0083,2.6972,0.0385,2.4057,2.4054,2.4083,2.4051,3.7038,2.4479,2.8411,2.4199
datatrieve,0.0057,1.8928,0.0236,1.9295,1.9292,1.9308,1.9289,2.0285,1.6726,2.2984,1.9345
Mean,0.0070,2.2950,0.0310,2.1676,2.1673,2.1695,2.1670,2.8661,2.0602,2.5698,2.1772
Rank,1.0000,6.5000,2.0000,6.0000,5.0000,7.0000,4.0000,10.5000,5.5000,10.5000,8.0000


In [6]:
ct.res_statistics(tables, params["metrics"], f"results/{params['exp_name']}", colors=params["calib_method_colors"])

metric acc
Friedman Test Statistic: 18.502415458937197
P-value: 0.047057326960468854
The differences between groups are significant.
metric brier
Friedman Test Statistic: 17.727272727272734
P-value: 0.05974215905780994
The differences between groups are significant.
metric ece
Friedman Test Statistic: 17.727272727272734
P-value: 0.05974215905780994
The differences between groups are significant.
metric logloss
Friedman Test Statistic: 19.0
P-value: 0.04026268234060996
The differences between groups are significant.
metric time
Friedman Test Statistic: 16.727272727272734
P-value: 0.08062080720213384
The differences between groups are significant.


In [7]:
ct.save_metrics_to_latex(tables, params["metrics"], params['exp_name'])

In [8]:
tables["acc"]

,RF_d,RF_opt,RF_large,Platt,ISO,Beta,VA,CT,CT_minleaf,PPA,Rank
Data,,,,,,,,,,,
cm1,0.891168,0.900000,0.894788,0.901616,0.900816,0.901616,0.900412,0.899212,0.901616,0.901616,0.901616
datatrieve,0.906154,0.910769,0.903077,0.915385,0.910769,0.915385,0.900000,0.907692,0.912308,0.912308,0.915385
Mean,0.898661,0.905385,0.898932,0.908500,0.905793,0.908500,0.900206,0.903452,0.906962,0.906962,0.908500
Rank,10.000000,7.250000,10.000000,2.500000,6.250000,2.500000,9.000000,8.500000,3.750000,3.750000,2.500000
